In [ ]:
import requests as req
import pandas as pd
import re

In [ ]:
 #conda install -c anaconda beautifulsoup4 

In [ ]:
res = req.get('https://www.accessdata.fda.gov/scripts/drugshortages/')

In [ ]:
B_URL = 'https://www.accessdata.fda.gov/scripts/drugshortages/'

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(res.content)

In [ ]:
divs = soup.find_all(['tbody'])

In [ ]:
divs = divs[0].find_all('tr')

In [ ]:
print(type(soup))
print(type(divs))
print(len(divs))

In [ ]:
status_l = []

In [ ]:
count = 0
while count < len(divs):
    for i in divs[count].find_all('strong'):
        status_l.append(i)
        count += 1
       # print('------------')
        #print(i)
        #print(test)
        #print('-----------')
        #print(type(i))

In [ ]:
status_f = []

In [ ]:
for i in status_l:
    w = i.string.strip()
    #print(i)
    status_f.append(w)

In [ ]:
###status_f

In [ ]:
href_l = []
title_l = []

In [ ]:
#filter out the second 'a' which included the information needed
count = 0
subcount = 0
while count < len(divs):
    for i in divs[count].find_all('a'):
            i.find_all('a')
            href = i.get('href')
            href_l.append(href)
            #print('href: ')
            #print(href)
            #print(count) 
            title = i.get('title')
            title_l.append(title)
            #print('title: ')
            #print(title)
            #print(count) 
            #print('\n')
            subcount +=1
    if subcount == 2:
            subcount = 0
            count += 1

In [ ]:
href_f = []

In [ ]:
#filter href_l
for i in range(1,len(divs)*2,2):
    words = href_l[i].split(' ')
    wordsjoin = '%20'.join(words)
    href_f.append(wordsjoin)
    #print(wordsjoin)
    #print(i)
 

In [ ]:
S_URL = 'https://www.accessdata.fda.gov/scripts/drugshortages/'

In [ ]:
href_link = []

In [ ]:
for i in href_f: 
    href_link.append(S_URL + i)

In [ ]:
###href_link

In [ ]:
title_f = []

In [ ]:
#filter title_l
for i in range(1,len(divs)*2,2):
    q = title_l[i]
    w = q.replace(' detail', '')
    title_f.append(re.sub('Link to ', '', w))   

In [ ]:
###title_f

In [ ]:
print(len(title_f))
print(len(href_link))
print(len(status_f))

In [ ]:
subs = {'Drug': [], 'Link':[],'Status':[]}

In [ ]:
subs['Drug'] = title_f
subs['Link'] = href_link
subs['Status'] = status_f

In [ ]:
#Create a new dataframe
sub_df = pd.DataFrame(subs, columns=['Drug','Link','Status'])

In [135]:
display(sub_df)

,Drug,Link,Status
0,Acetazolamide Injection,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
1,Amifostine Injection,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
2,Amino Acids,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
3,Amoxapine Tablets,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
4,Amphetamine Aspartate; Amphetamine Sulfate; De...,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
...,...,...,...
163,Varenicline Tartrate (Chantix) Tablets,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
164,Vecuronium Bromide for Injection,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
165,Vitamin A Palmitate (Aquasol A) Injection,https://www.accessdata.fda.gov/scripts/drugsho...,Currently in Shortage
166,Zinc Acetate Capsules,https://www.accessdata.fda.gov/scripts/drugsho...,Resolved


In [ ]:
#pip install mysql-connector-python
sub_df.info()

In [ ]:
#pip install PyMySQL

In [117]:
import pymysql

# Connect to the database
cnx = pymysql.connect(host='localhost',
                         user='root',
                         password='',
                         db='fda')
# create cursor
cursor=cnx.cursor()
#cnx.close()

In [118]:
db_Info = cnx.get_server_info()
print(db_Info)
#cursor.execute("CREATE TABLE shortages (Drug VARCHAR(255), Link VARCHAR(255), Status VARCHAR(255))")

8.0.28


In [121]:
cursor.execute("ALTER TABLE shortages ADD PRIMARY KEY (Drug)")



0

In [123]:
cursor.execute('DESCRIBE shortages')
for x in cursor:
    print(x)

('Drug', 'varchar(255)', 'NO', 'PRI', None, '')
('Link', 'varchar(255)', 'YES', '', None, '')
('Status', 'varchar(255)', 'YES', '', None, '')


In [131]:
#cursor.execute("DELETE FROM shortages WHERE Status = 'Currently in Shortage'")
#cnx.commit()

In [127]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in sub_df[0:21].columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in sub_df[0:21].iterrows():
    sql = "INSERT INTO `shortages` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    #print(sql)
    #print(row)

    # the connection is not autocommitted by default, so we must commit to save our changes
    cnx.commit()

IntegrityError: (1062, "Duplicate entry 'Acetazolamide Injection' for key 'shortages.PRIMARY'")

In [ ]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in sub_df[0:21].columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in sub_df[0:21].iterrows():
    sql = "INSERT INTO `shortages` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    #print(sql)
    #print(row)

    # the connection is not autocommitted by default, so we must commit to save our changes
    cnx.commit()

In [125]:
cursor.execute("SELECT * FROM shortages WHERE Status = 'resolved'")
for x in cursor:
    print(x)

('Anagrelide Hydrochloride Capsules', 'https://www.accessdata.fda.gov/scripts/drugshortages/dsp_ActiveIngredientDetails.cfm?AI=Anagrelide%20Hydrochloride%20Capsules&st=r&tab=tabs-1', 'Resolved')
('Atropine Sulfate Ophthalmic Ointment', 'https://www.accessdata.fda.gov/scripts/drugshortages/dsp_ActiveIngredientDetails.cfm?AI=Atropine%20Sulfate%20Ophthalmic%20Ointment&st=r&tab=tabs-1', 'Resolved')
('Calcitriol Injection 1MCG/ML', 'https://www.accessdata.fda.gov/scripts/drugshortages/dsp_ActiveIngredientDetails.cfm?AI=Calcitriol%20Injection%201MCG/ML&st=r&tab=tabs-1', 'Resolved')


In [ ]:
#from pymongo import MongoClient

In [ ]:
#client = MongoClient()
#db = client['CDC']
#collection = db['short']

In [ ]:
#sub_df.to_dict('records')

In [ ]:
#collection.insert_many(sub_df.to_dict('records'))